In [6]:
import sklearn
import pandas as pd
import numpy as np
#import scikitplot as skplt
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, make_scorer

from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.preprocessing import normalize
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report

from imblearn.over_sampling import SMOTE

In [90]:
df = pd.read_csv('../data/train.csv', '|')

X, y = df.drop('fraud', axis=1), df['fraud']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

sm = SMOTE(random_state=123)
X_res, y_res = sm.fit_resample(X_train, y_train)

In [3]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

detailed param grid

In [77]:
[int(x) for x in np.linspace(start = 1300, stop = 1900, num = 10)]

[1300, 1366, 1433, 1500, 1566, 1633, 1700, 1766, 1833, 1900]

In [78]:
[int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

[200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]

In [37]:
def pf(y,y_pred):
    y_true = y
    confusion_m = np.array(confusion_matrix(y_true, y_pred))
    price_matrix = [[0,-25],[-5,5]]
    profit_matrix = np.matmul(confusion_m,price_matrix)
    return profit_matrix.sum()

In [38]:
clf = RandomForestClassifier()
clf_random = RandomizedSearchCV(estimator = clf, 
                               param_distributions = random_grid, 
                               n_iter = 200, 
                               cv = 5, 
                               verbose=5, 
                               random_state=42, 
                               n_jobs = -1,
                               scoring = make_scorer(pf, greater_is_better=True),
#                                 scoring = make_scorer()
                               return_train_score = True)
clf_random.fit(X_res, y_res)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:   34.7s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 264 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed:  7.1min
[Parallel(n_jobs=-1)]: Done 858 tasks      | elapsed:  9.8min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed: 11.3min finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid='warn', n_iter=200, n_jobs=-1,
          param_distributions={'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score=True, scoring=make_scorer(pf), verbose=

In [39]:
clf_random.best_params_

{'n_estimators': 1600,
 'min_samples_split': 5,
 'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'max_depth': 10,
 'bootstrap': False}

In [92]:
clf2 = RandomForestClassifier(bootstrap=False,
                              max_depth=10,
                              max_features='sqrt',
                              min_samples_leaf=2,
                              min_samples_split=5,
                              n_estimators=1600)

clf2.fit(X_res, y_res)

pred = clf2.predict(X_res)
acc = clf2.score(X_res, y_res)
prec = precision_score(y_res, pred)
rec = recall_score(y_res, pred)
cost = pf(y_res, pred)

print('Accuracy: {}, precision: {}, recall: {}, cost: {}'.format(acc, prec, rec, cost))

Accuracy: 0.994378074490513, precision: 0.9895615866388309, recall: 0.9992972593113141, cost: -35225


detailed param grid

In [82]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 1300, stop = 1900, num = 10)]
# Number of features to consider at every split
max_features = ['sqrt']
# Maximum number of levels in tree
max_depth = ([int(x) for x in np.linspace(5, 15, num = 11)])
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [3,4,5,6,7]
# Minimum number of samples required at each leaf node
min_samples_leaf = [2, 3]
# Method of selecting samples for training each tree
bootstrap = [False]

# Create the random grid
detail_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [84]:
grid_clf = GridSearchCV(estimator = clf, param_grid = detail_grid, 
                          cv = 5, n_jobs = -1, verbose = 5)
grid_clf.fit(X_res,y_res)

Fitting 5 folds for each of 1200 candidates, totalling 6000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:   34.7s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 264 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed:  7.2min
[Parallel(n_jobs=-1)]: Done 858 tasks      | elapsed: 10.0min
[Parallel(n_jobs=-1)]: Done 1128 tasks      | elapsed: 13.4min
[Parallel(n_jobs=-1)]: Done 1434 tasks      | elapsed: 17.4min
[Parallel(n_jobs=-1)]: Done 1776 tasks      | elapsed: 22.1min
[Parallel(n_jobs=-1)]: Done 2154 tasks      | elapsed: 27.7min
[Parallel(n_jobs=-1)]: Done 2568 tasks      | elapsed: 34.4min
[Parallel(n_jobs=-1)]: Done 3018 tasks      | elapsed: 42.0min
[Parallel(n_jobs=-1)]: Done 3504 tasks      | elapsed: 50.5min
[Parallel(n_jobs=-1)]: Done 4026 tasks      | elapsed: 59.4min
[Parallel(n_jobs=-1)]: Done 4584 tasks      | e

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [1300, 1366, 1433, 1500, 1566, 1633, 1700, 1766, 1833, 1900], 'max_features': ['sqrt'], 'max_depth': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, None], 'min_samples_split': [3, 4, 5, 6, 7], 'min_samples_leaf': [2, 3], 'bootstrap': [False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=5)

In [85]:
grid_clf.best_params_

{'bootstrap': False,
 'max_depth': None,
 'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'min_samples_split': 3,
 'n_estimators': 1300}

In [91]:
clf3 = RandomForestClassifier(bootstrap=False,
                              max_depth=None,
                              max_features='sqrt',
                              min_samples_leaf=2,
                              min_samples_split=3,
                              n_estimators=1300)

clf3.fit(X_res, y_res)

pred = clf3.predict(X_res)
acc = clf3.score(X_res, y_res)
prec = precision_score(y_res, pred)
rec = recall_score(y_res, pred)
cost = pf(y_res, pred)

print('Accuracy: {}, precision: {}, recall: {}, cost: {}'.format(acc, prec, rec, cost))

Accuracy: 1.0, precision: 1.0, recall: 1.0, cost: -35575
